### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [30]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [31]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [32]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn
tfidfvect = TfidfVectorizer()

In [33]:
# en el atributo `data` accedemos al texto
newsgroups_train.data[0]

'I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.'

In [34]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [35]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'cantidad de documentos: {X_train.shape[0]}')
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
cantidad de documentos: 11314
tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [36]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [37]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [38]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [39]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [40]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [41]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [42]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [43]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([4811, 6635, 4253, ..., 1911, 1825, 1828])

In [44]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [45]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [46]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [47]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [48]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [49]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

np.float64(0.5854345727938506)

### Consigna del desafío 1

# Resolución - Desafio 1


## Punto 1

Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

In [50]:
# se seleccionar 5 documentos al azar con semiilla 43
np.random.seed(55)
idxs = np.random.choice(X_train.shape[0], 5)
print(idxs)

# calcular la similaridad coseno de cada documento seleccionado con el resto de los documentos y guardar los 5 más similares por cada uno. Junto con la similitud coseno, guardar el índice de los documentos más similares.
mostsim = {}
mostsim_cossim = {}

for idx in idxs:
    cossim = cosine_similarity(X_train[idx], X_train)[0]
    mostsim[idx] = np.argsort(cossim)[::-1][1:6]
    mostsim_cossim[idx] = np.sort(cossim)[::-1][1:6]


[ 4762   968 10213  8865  6198]


A continuación se analizan las similitudes de los documentos seleccionados con el resto de los documentos del dataset.

In [51]:
# mostrar la clase de cada documento seleccionado y de los 5 más similares, agregando la similitud coseno

for idx in idxs:
    print(f'Documento seleccionado {idx}: {newsgroups_train.target_names[y_train[idx]]}')
    for i, sim in zip(mostsim[idx], mostsim_cossim[idx]):
        print(f'\t Documento similar {i}: {newsgroups_train.target_names[y_train[i]]} - Similitud coseno: {sim:.2f}')

Documento seleccionado 4762: talk.politics.guns
	 Documento similar 6894: talk.politics.guns - Similitud coseno: 0.31
	 Documento similar 7333: talk.politics.guns - Similitud coseno: 0.28
	 Documento similar 4271: talk.politics.misc - Similitud coseno: 0.28
	 Documento similar 1191: talk.politics.misc - Similitud coseno: 0.27
	 Documento similar 5895: talk.politics.guns - Similitud coseno: 0.26
Documento seleccionado 968: rec.sport.baseball
	 Documento similar 5234: rec.sport.baseball - Similitud coseno: 0.44
	 Documento similar 7528: rec.sport.baseball - Similitud coseno: 0.19
	 Documento similar 1326: rec.sport.baseball - Similitud coseno: 0.19
	 Documento similar 2510: rec.sport.baseball - Similitud coseno: 0.19
	 Documento similar 8759: rec.sport.hockey - Similitud coseno: 0.18
Documento seleccionado 10213: rec.autos
	 Documento similar 2697: rec.autos - Similitud coseno: 0.29
	 Documento similar 1777: rec.autos - Similitud coseno: 0.23
	 Documento similar 7788: rec.autos - Similit

Como se puede observar, aunque las similitudes son bajas, las clases de los documentos seleccionados y los documentos más similares presentan leves diferencias. 
Sin embargo en los documentos 8865 y 6198, las clases difieren en mayor medida. Por este motivo, estos documentos se analizarán en mayor profundidad.

In [52]:
print(f'Documento seleccionado {8865}')
print(f'{newsgroups_train.data[8865]}')
for i in mostsim[8865]:
    print('\n#################\n')
    print(f'Documento similar {i}')
    print(f'{newsgroups_train.data[i]}')
    

Documento seleccionado 8865
If you're willing to do a little work, you can make DrawnButtons do what you
want, more-or-less. One of my colleagues here at GE CRD has done just that
in our internal LYMB system.

We have a matrix transform class that makes it easy to compute a series of
dial positions from a single set of vectors. Each set of vectors is then
drawn into a pixmap. Clicking the button advances the knob's state and
changes to the next pixmap in the sequence.

Using DrawnButtons obviously still constrains you to taking up a rectangular
portion of the parent widget, but that's normally not a big shortcoming. You
can make things look circular enough.


#################

Documento similar 4614
Archive-name: x-faq/part5
Last-modified: 1993/04/04

----------------------------------------------------------------------
Subject: 119)  I'm writing a widget and can't use a float as a resource value.

Float resources are not portable; the size of the value may be larger than
the size of

In [53]:
print(f'Documento seleccionado {6198}')
print(f'{newsgroups_train.data[6198]}')
for i in mostsim[6198]:
    print('\n#################\n')
    print(f'Documento similar {i}')
    print(f'{newsgroups_train.data[i]}')

Documento seleccionado 6198

Well, the fact of the matter is that poverty is imperfectly related to 
social and political instability, while economic inequality is much more
strongly related.  In virtually all major revolutions including
England (the Puritan revolution), France, Russia and China, the 
revolutions occurred as economies were undergoing substantial long term 
growth and poverty was declining.  What sets off revolutions is massive
inequality coupled with a perception on the part of those at the bottom
that social change is possible.

If "poverty (were) the main engine of social instability," this typical
historical pattern would not hold.  In fact, revolutions would have been
far more typical before the nineteenth century than since that time.

[Much deleted...]


Sorry, buddy, but some other "blowhards" managed to include the "general
welfare" in another portion of the constitution.

Article I Section 8: "The Congress shall have the power to lay and collect
taxes...to pay

Luego de la lectura de los documentos presentados anteriormente y el análisis de las clases resultandes de los documentos más similares, se puede concluir que la similaridad de los documentos seleccionados con los documentos más similares no es significativa en la mayoría de los casos.

Sin embargo, se pueden llegar a encontrar relaciones en menor o mayor medida entre los documentos seleccionados y los documentos más similares. 
Por ejemplo, en el caso del documento 6198, se puede observar que el contenido del documento es similar a los documentos más similares, aunque la clase de los documentos difiere. Por otro lado, en el caso del documento 8865, el contenido del documento no es similar a los documentos más similares, y la clase de los documentos también difiere.

## Punto 2

Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.


In [54]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, f1_score

In [55]:
tfidf_params = {
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],  # Expandimos ngram_range
    'tfidf__min_df': [1, 5, 10],  # Agregamos más opciones para min_df
    'tfidf__max_df': [0.7, 0.8, 0.9],  # Agregamos más opciones para max_df
    'tfidf__use_idf': [True, False],  # Incluimos el uso de idf
    'tfidf__smooth_idf': [True, False],  # Incluimos la suavización de idf
}

models = {
    'MultinomialNB': MultinomialNB(),
    'ComplementNB': ComplementNB(),
}

# Usamos GridSearchCV con un pipeline para optimizar tanto el vectorizador como el modelo
best_model = None
best_score = 0
best_params = {}

for model_name, model in models.items():
    # Creamos un pipeline con el vectorizador y el modelo
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer()),
        ('model', model)
    ])
    
    # Definimos el grid de búsqueda
    param_grid = {**tfidf_params}  # Solo exploramos parámetros del tfidf en este caso

    # Definimos el scorer como f1 macro
    scorer = make_scorer(f1_score, average='macro')

    # Usamos GridSearchCV para buscar los mejores parámetros
    grid_search = GridSearchCV(pipeline, param_grid, scoring=scorer, cv=5, n_jobs=-1)
    grid_search.fit(newsgroups_train.data, y_train)

    # Guardamos el mejor resultado si mejora el score
    if grid_search.best_score_ > best_score:
        best_score = grid_search.best_score_
        best_model = model_name
        best_params = grid_search.best_params_

# Mostramos los resultados
print(f"Mejor modelo: {best_model} con f1-score {best_score}")
print(f"Mejores parámetros: {best_params}")

Mejor modelo: ComplementNB con f1-score 0.7530037078754678
Mejores parámetros: {'tfidf__max_df': 0.8, 'tfidf__min_df': 1, 'tfidf__ngram_range': (1, 1), 'tfidf__smooth_idf': False, 'tfidf__use_idf': True}


## Punto 3

Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.

In [25]:
tfidfvect = TfidfVectorizer(ngram_range=(1, 1), min_df=1, max_df=0.8, use_idf=True, smooth_idf=False)

In [26]:
X_train = tfidfvect.fit_transform(newsgroups_train.data)

In [27]:
X_train_transpose = X_train.T

In [28]:
words = tfidfvect.get_feature_names_out()

In [29]:
# Seleccionar manualmente 5 palabras
selected_words = ['windows', 'car', 'fbi', 'electronic', 'country']  # Reemplaza con tus palabras seleccionadas

# Calcular la similitud del coseno entre las palabras
similarity_matrix = cosine_similarity(X_train_transpose)

# Encontrar las 5 palabras más similares para cada una de las palabras seleccionadas
for word in selected_words:
    if word in words:
        word_index = np.where(words == word)[0][0]
        similar_indices = similarity_matrix[word_index].argsort()[-6:-1][::-1]  # Excluir la palabra en sí misma
        similar_words = [words[i] for i in similar_indices]
        print(f"Palabras más similares a '{word}': {similar_words}")
    else:
        print(f"La palabra '{word}' no se encuentra en el vocabulario.")

Palabras más similares a 'windows': ['dos', 'ms', 'microsoft', 'nt', 'for']
Palabras más similares a 'car': ['cars', 'criterium', 'civic', 'owner', 'dealer']
Palabras más similares a 'fbi': ['koresh', 'batf', '3691', 'permitter', 'defuse']
Palabras más similares a 'electronic': ['wichever', 'towwang', 'caen', 'cruptology', 'fluourescent']
Palabras más similares a 'country': ['borden', 'critizises', '1993apr25', 'antisemite', 'missusing']


Si bien las palabras windows y car, tienen una alta similaridad con las palabras más similares, las palabras seleccionadas en los otros casos no presentan similitudes significativas con las palabras más similares.

Por ejemplo, en el caso de la palabra "car", las palabras más similares hacen referencia a vehículos directa o indirectamente, lo cual tiene sentido. Por otro lado, en la palabra "country", las palabras más similares no presentan una relación alguna con la palabra seleccionada.